<a href="https://colab.research.google.com/github/CarolinaOliveira001/SupremeCourtJudgementPrediction/blob/main/secondTry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer, PorterStemmer

In [4]:
df = pd.read_csv("justice.csv")
df = df[["facts", "issue_area", "first_party_winner"]]
df.head(5)

,facts,issue_area,first_party_winner
0,"<p>In 1970, Jane Roe (a fictional name used in...",NaN,True
1,<p>Joan Stanley had three children with Peter ...,Civil Rights,True
2,<p>John Giglio was convicted of passing forged...,Due Process,True
3,"<p>The Idaho Probate Code specified that ""male...",Civil Rights,True
4,"<p>Miller, after conducting a mass mailing cam...",First Amendment,True


In [5]:
df['issue_area'] = df['issue_area'].replace({"Due Process": "Others", "Privacy": "Others", "Federal Taxation": "Others", 
                                                     "Unions": "Others", "Federalism": "Others", "Attorneys": "Others",
                                                     "Miscellaneous": "Others", "Interstate Relations": "Others", "Private Action": "Others"})

In [6]:
df.isna().sum()

facts                   0
issue_area            142
first_party_winner     15
dtype: int64

In [7]:
df.dropna(subset=['first_party_winner'], inplace=True)

In [8]:
df.isna().sum()

facts                   0
issue_area            129
first_party_winner      0
dtype: int64

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [10]:
le.fit(df["first_party_winner"])

LabelEncoder()

In [11]:
df["first_party_winner"] = le.transform(df["first_party_winner"])

In [12]:
df_withNA = df[df['issue_area'].isna()]
df_withNA = df_withNA.reset_index(drop=True)

In [13]:
df_withoutNA = df.dropna(subset=['issue_area'])
df_withoutNA = df_withoutNA.reset_index(drop=True)

In [14]:
print("There are", df_withoutNA['facts'].apply(lambda x: len(x.split(' '))).sum(), "words in the dataframe.")

There are 549349 words in the dataframe.


In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
stop_words = stopwords.words("english")
wordnet_lemmatizer = WordNetLemmatizer()

In [17]:
pattern = r'[' + string.punctuation + ']'

def clean_text(text, stemming = False, lemmatization = False):
    """
        text: a string
        
        return: modified initial string
    """
    text = re.sub(pattern, " ", str(text))
    text = text.lower()
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(' +', ' ', text)
    text = ' '.join(word for word in text.split() if word not in stop_words) # delete stopwors from text
    if stemming:
        st = PorterStemmer()
        text = " ".join([st.stem(w) for w in text.split()])
    
    if lemmatization:
        wordnet_lemmatizer = WordNetLemmatizer()
        text = ' '.join([wordnet_lemmatizer.lemmatize(w, pos='v') for w in text.split()])
    return text
    

In [18]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [19]:
df_withoutNA['facts'] = df_withoutNA['facts'].map(lambda x: clean_text(x, stemming=False, lemmatization = True))

In [20]:
df_withoutNA.head(3)

,facts,issue_area,first_party_winner
0,joan stanley three children peter stanley stan...,Civil Rights,1
1,john giglio convict pass forge money order app...,Others,1
2,idaho probate code specify males must prefer f...,Civil Rights,1


In [21]:
print("There are",df_withoutNA['facts'].apply(lambda x: len(x.split(' '))).sum(), "words in the dataframe.")

There are 322095 words in the dataframe.


In [41]:
my_issue_area = list(set(df_withoutNA["issue_area"]))

# Without deleting most frequent words

In [ ]:
X = df_withoutNA.facts
y = df_withoutNA.issue_area
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [ ]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

CPU times: user 105 ms, sys: 0 ns, total: 105 ms
Wall time: 117 ms


In [ ]:
list(set(y_pred))

['Economic Activity',
 'Others',
 'First Amendment',
 'Criminal Procedure',
 'Civil Rights']

In [ ]:
list(set(y_test))

['Economic Activity',
 'Others',
 'First Amendment',
 'Criminal Procedure',
 'Judicial Power',
 'Civil Rights']

In [ ]:
import warnings
warnings.filterwarnings('always') #We can set it later to 'ignore'.

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_issue_area))

accuracy 0.5580168776371308
                    precision    recall  f1-score   support

            Others       0.60      0.51      0.55       184
 Economic Activity       0.51      0.99      0.67       258
   First Amendment       0.64      0.63      0.64       161
Criminal Procedure       0.84      0.17      0.28        96
      Civil Rights       0.00      0.00      0.00       111
    Judicial Power       0.56      0.45      0.50       138

          accuracy                           0.56       948
         macro avg       0.53      0.46      0.44       948
      weighted avg       0.53      0.56      0.50       948



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [ ]:
%%time
y_pred = sgd.predict(X_test)

CPU times: user 220 ms, sys: 64 ms, total: 284 ms
Wall time: 224 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.6867088607594937
                    precision    recall  f1-score   support

            Others       0.74      0.65      0.69       184
 Economic Activity       0.68      0.98      0.81       258
   First Amendment       0.69      0.71      0.70       161
Criminal Procedure       0.68      0.72      0.70        96
      Civil Rights       0.62      0.21      0.31       111
    Judicial Power       0.64      0.52      0.57       138

          accuracy                           0.69       948
         macro avg       0.68      0.63      0.63       948
      weighted avg       0.68      0.69      0.66       948



In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, solver='lbfgs', max_iter=100000)),
               ])
logreg.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 LogisticRegression(C=100000.0, max_iter=100000, n_jobs=1))])

In [ ]:
%%time
y_pred = logreg.predict(X_test)

CPU times: user 203 ms, sys: 614 µs, total: 204 ms
Wall time: 205 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.7056962025316456
                    precision    recall  f1-score   support

            Others       0.72      0.69      0.71       184
 Economic Activity       0.78      0.94      0.85       258
   First Amendment       0.70      0.70      0.70       161
Criminal Procedure       0.70      0.77      0.74        96
      Civil Rights       0.53      0.31      0.39       111
    Judicial Power       0.62      0.58      0.60       138

          accuracy                           0.71       948
         macro avg       0.67      0.66      0.66       948
      weighted avg       0.69      0.71      0.69       948



In [ ]:
from sklearn.ensemble import RandomForestClassifier

randfor = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators= 10, max_depth=10, random_state=10)),
               ])
randfor.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 RandomForestClassifier(max_depth=10, n_estimators=10,
                                        random_state=10))])

In [ ]:
%%time
y_pred = randfor.predict(X_test)

CPU times: user 188 ms, sys: 973 µs, total: 189 ms
Wall time: 194 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.4272151898734177
                    precision    recall  f1-score   support

            Others       0.60      0.32      0.42       184
 Economic Activity       0.38      0.97      0.54       258
   First Amendment       0.46      0.35      0.39       161
Criminal Procedure       0.88      0.23      0.36        96
      Civil Rights       0.57      0.04      0.07       111
    Judicial Power       0.48      0.11      0.18       138

          accuracy                           0.43       948
         macro avg       0.56      0.33      0.33       948
      weighted avg       0.52      0.43      0.37       948



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
graboos = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', GradientBoostingClassifier(n_estimators=10, max_depth=10, random_state=10)),
               ])
graboos.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 GradientBoostingClassifier(max_depth=10, n_estimators=10,
                                            random_state=10))])

In [ ]:
%%time
y_pred = graboos.predict(X_test)

CPU times: user 195 ms, sys: 1.8 ms, total: 197 ms
Wall time: 197 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.6086497890295358
                    precision    recall  f1-score   support

            Others       0.63      0.54      0.58       184
 Economic Activity       0.70      0.88      0.78       258
   First Amendment       0.63      0.60      0.62       161
Criminal Procedure       0.68      0.64      0.66        96
      Civil Rights       0.47      0.28      0.35       111
    Judicial Power       0.40      0.44      0.42       138

          accuracy                           0.61       948
         macro avg       0.58      0.56      0.57       948
      weighted avg       0.60      0.61      0.60       948



In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [ ]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [ ]:
import xgboost as xgb

xgboo = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', xgb.XGBClassifier()),
               ])
xgboo.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', XGBClassifier(objective='multi:softprob'))])

In [ ]:
%%time
y_pred = xgboo.predict(X_test)

CPU times: user 279 ms, sys: 6.79 ms, total: 286 ms
Wall time: 285 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.6350210970464135
                    precision    recall  f1-score   support

            Others       0.67      0.60      0.63       184
 Economic Activity       0.74      0.89      0.81       258
   First Amendment       0.63      0.61      0.62       161
Criminal Procedure       0.63      0.70      0.66        96
      Civil Rights       0.47      0.27      0.34       111
    Judicial Power       0.45      0.47      0.46       138

          accuracy                           0.64       948
         macro avg       0.60      0.59      0.59       948
      weighted avg       0.62      0.64      0.62       948



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

kn = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('kn',KNeighborsClassifier(n_neighbors=3)),
               ])
kn.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('kn', KNeighborsClassifier(n_neighbors=3))])

In [ ]:
%%time
y_pred = kn.predict(X_test)

CPU times: user 384 ms, sys: 1.24 ms, total: 386 ms
Wall time: 388 ms


In [ ]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.5928270042194093
                    precision    recall  f1-score   support

            Others       0.50      0.70      0.58       184
 Economic Activity       0.66      0.81      0.73       258
   First Amendment       0.63      0.60      0.61       161
Criminal Procedure       0.65      0.48      0.55        96
      Civil Rights       0.50      0.23      0.32       111
    Judicial Power       0.59      0.40      0.47       138

          accuracy                           0.59       948
         macro avg       0.59      0.54      0.54       948
      weighted avg       0.59      0.59      0.58       948



# With deleting some words

In [ ]:
bunchOfWords = "".join(df_withoutNA["facts"].tolist())

We will perform a bag of words and try to find the words that appear 1000

In [ ]:
data_corpus = set()
for word in bunchOfWords.split(" "):
    if word not in data_corpus: 
        data_corpus.add(word)
data_corpus=sorted(data_corpus)

In [ ]:
def convert(set):
    return [*set, ]

data_corpus = convert(data_corpus)

In [ ]:
one_hot_encoding = []
split = bunchOfWords.split(" ")
for word in data_corpus:
    count = split.count(word) 
    if word in split:
        one_hot_encoding.append(count)
    else:
        one_hot_encoding.append(count)

In [ ]:
dfWords = pd.DataFrame({"words": data_corpus, "numberOfFrequency": one_hot_encoding})
dfWords["numberOfFrequency"] = pd.to_numeric(dfWords["numberOfFrequency"])

In [ ]:
irrelevantWords = dfWords[dfWords["numberOfFrequency"] > 1000]
irrelevantWords = set(irrelevantWords["words"])

In [ ]:
def more_clean_text(text):
    text = ' '.join(word for word in text.split() if word not in irrelevantWords) # delete stopwors from text
    return text

In [ ]:
df_withoutNA['facts'] = df_withoutNA['facts'].apply(more_clean_text)

In [ ]:
print("There are",df_withoutNA['facts'].apply(lambda x: len(x.split(' '))).sum(), "words in the dataframe.")

There are 267703 words in the dataframe.


In [ ]:
X = df_withoutNA.facts
y = df_withoutNA.issue_area
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [ ]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

CPU times: user 166 ms, sys: 1.7 ms, total: 167 ms
Wall time: 170 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_issue_area))

accuracy 0.520042194092827
                    precision    recall  f1-score   support

            Others       0.69      0.48      0.57       184
 Economic Activity       0.43      1.00      0.60       258
   First Amendment       0.63      0.61      0.62       161
Criminal Procedure       1.00      0.07      0.14        96
      Civil Rights       0.00      0.00      0.00       111
    Judicial Power       0.64      0.30      0.41       138

          accuracy                           0.52       948
         macro avg       0.57      0.41      0.39       948
      weighted avg       0.55      0.52      0.45       948



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [ ]:
%%time
y_pred = sgd.predict(X_test)

CPU times: user 215 ms, sys: 68.6 ms, total: 283 ms
Wall time: 217 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.6835443037974683
                    precision    recall  f1-score   support

            Others       0.76      0.65      0.70       184
 Economic Activity       0.66      0.98      0.79       258
   First Amendment       0.67      0.71      0.69       161
Criminal Procedure       0.73      0.72      0.72        96
      Civil Rights       0.60      0.19      0.29       111
    Judicial Power       0.65      0.51      0.57       138

          accuracy                           0.68       948
         macro avg       0.68      0.63      0.63       948
      weighted avg       0.68      0.68      0.66       948



In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, solver='lbfgs', max_iter=100000)),
               ])
logreg.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 LogisticRegression(C=100000.0, max_iter=100000, n_jobs=1))])

In [ ]:
%%time
y_pred = logreg.predict(X_test)

CPU times: user 165 ms, sys: 15.7 ms, total: 180 ms
Wall time: 167 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.7046413502109705
                    precision    recall  f1-score   support

            Others       0.74      0.68      0.71       184
 Economic Activity       0.75      0.94      0.83       258
   First Amendment       0.68      0.71      0.70       161
Criminal Procedure       0.70      0.76      0.73        96
      Civil Rights       0.55      0.29      0.38       111
    Judicial Power       0.65      0.59      0.62       138

          accuracy                           0.70       948
         macro avg       0.68      0.66      0.66       948
      weighted avg       0.69      0.70      0.69       948



In [ ]:
from sklearn.ensemble import RandomForestClassifier

randfor = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators= 10, max_depth=10, random_state=10)),
               ])
randfor.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 RandomForestClassifier(max_depth=10, n_estimators=10,
                                        random_state=10))])

In [ ]:
%%time
y_pred = randfor.predict(X_test)

CPU times: user 176 ms, sys: 3.97 ms, total: 180 ms
Wall time: 182 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.4208860759493671
                    precision    recall  f1-score   support

            Others       0.55      0.32      0.40       184
 Economic Activity       0.39      0.98      0.56       258
   First Amendment       0.43      0.35      0.39       161
Criminal Procedure       0.75      0.16      0.26        96
      Civil Rights       0.57      0.04      0.07       111
    Judicial Power       0.34      0.08      0.13       138

          accuracy                           0.42       948
         macro avg       0.51      0.32      0.30       948
      weighted avg       0.48      0.42      0.35       948



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
graboos = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', GradientBoostingClassifier(n_estimators=10, max_depth=10, random_state=10)),
               ])
graboos.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 GradientBoostingClassifier(max_depth=10, n_estimators=10,
                                            random_state=10))])

In [ ]:
%%time
y_pred = graboos.predict(X_test)

CPU times: user 174 ms, sys: 147 µs, total: 174 ms
Wall time: 175 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.6054852320675106
                    precision    recall  f1-score   support

            Others       0.64      0.55      0.59       184
 Economic Activity       0.70      0.87      0.77       258
   First Amendment       0.60      0.57      0.59       161
Criminal Procedure       0.65      0.57      0.61        96
      Civil Rights       0.53      0.27      0.36       111
    Judicial Power       0.41      0.52      0.46       138

          accuracy                           0.61       948
         macro avg       0.59      0.56      0.56       948
      weighted avg       0.60      0.61      0.60       948



In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [ ]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [ ]:
import xgboost as xgb

xgboo = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', xgb.XGBClassifier()),
               ])
xgboo.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', XGBClassifier(objective='multi:softprob'))])

In [ ]:
%%time
y_pred = xgboo.predict(X_test)

CPU times: user 272 ms, sys: 4.83 ms, total: 277 ms
Wall time: 276 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.6244725738396625
                    precision    recall  f1-score   support

            Others       0.64      0.55      0.59       184
 Economic Activity       0.73      0.91      0.81       258
   First Amendment       0.62      0.58      0.60       161
Criminal Procedure       0.67      0.65      0.66        96
      Civil Rights       0.47      0.29      0.36       111
    Judicial Power       0.44      0.50      0.47       138

          accuracy                           0.62       948
         macro avg       0.59      0.58      0.58       948
      weighted avg       0.62      0.62      0.61       948



In [ ]:
kn = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', KNeighborsClassifier(n_neighbors=3)),
               ])
kn.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', KNeighborsClassifier(n_neighbors=3))])

In [ ]:
%%time
y_pred = kn.predict(X_test)

CPU times: user 315 ms, sys: 2.35 ms, total: 318 ms
Wall time: 322 ms


In [ ]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.5780590717299579
                    precision    recall  f1-score   support

            Others       0.47      0.66      0.55       184
 Economic Activity       0.67      0.80      0.73       258
   First Amendment       0.59      0.60      0.60       161
Criminal Procedure       0.66      0.50      0.57        96
      Civil Rights       0.45      0.18      0.26       111
    Judicial Power       0.54      0.41      0.46       138

          accuracy                           0.58       948
         macro avg       0.56      0.52      0.53       948
      weighted avg       0.57      0.58      0.56       948



# Word2Vec

In [ ]:
pip install --upgrade tensorflow-hub

In [ ]:
pip install tensorflow_text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_withoutNA["facts"],df_withoutNA["issue_area"],test_size=0.2,shuffle=True)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [ ]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2, batch_size = 32)

Epoch 1/2
79/79 [==============================] - 1090s 13s/step - loss: -13.0583 - accuracy: 0.2731
Epoch 2/2
79/79 [==============================] - 1056s 13s/step - loss: -40.5781 - accuracy: 0.2762


In [ ]:
y_train

array([1, 3, 2, ..., 0, 1, 3])

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
print(y_predicted)

20/20 [==============================] - 259s 13s/step
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [ ]:
print('accuracy %s' % accuracy_score(y_predicted, y_test))
print(classification_report(y_test, y_predicted,target_names=my_issue_area))

accuracy 0.254746835443038
                    precision    recall  f1-score   support

            Others       0.00      0.00      0.00       113
    Judicial Power       0.25      1.00      0.41       161
Criminal Procedure       0.00      0.00      0.00       104
 Economic Activity       0.00      0.00      0.00        70
      Civil Rights       0.00      0.00      0.00        69
   First Amendment       0.00      0.00      0.00       115

          accuracy                           0.25       632
         macro avg       0.04      0.17      0.07       632
      weighted avg       0.06      0.25      0.10       632



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Issue Area prediction

In [23]:
second_df = pd.read_csv("us_classactions_verdicts.csv")
second_df = second_df[["target_text", "verdict"]]

In [24]:
second_df.rename(columns = {'target_text':'facts', 'verdict': 'first_party_winner'}, inplace = True)

In [25]:
second_df.head(3)

,facts,first_party_winner
0,Knowing and/or Willful Violation of the Telep...,lose
1,17. Defendants operate an electrical construc...,win
2,(Fair Labor Standards Act Violations) (Viola...,win


In [26]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [27]:
le.fit(second_df["first_party_winner"])

LabelEncoder()

In [28]:
second_df["first_party_winner"] = le.transform(second_df["first_party_winner"])

In [29]:
second_df['facts'] = second_df['facts'].map(lambda x: clean_text(x, stemming=False, lemmatization = True))
df_withNA['facts'] = df_withNA['facts'].map(lambda x: clean_text(x, stemming=False, lemmatization = True))

In [30]:
df_withNA = df_withNA[["facts", "first_party_winner"]]

In [31]:
df_toPredict = pd.concat([df_withNA, second_df])
df_toPredict = df_toPredict.reset_index(drop=True)

In [32]:
df_toPredict

,facts,first_party_winner
0,jane roe fictional name use court document pro...,1
1,wiley bolden residents mobile alabama bring cl...,1
2,county los angeles impose ad valorem tax manuf...,0
3,rincon island artificial island coast ventura ...,1
4,president jimmy carter act without congression...,0
...,...,...
3124,fair labor standards act violations violations...,1
3125,guidelines regularly test user accessibility b...,0
3126,action bring class action plaintiff bring acti...,1
3127,discovery may reveal transmission additional f...,0


In [36]:
X = df_withoutNA.facts
y = df_withoutNA.issue_area
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, solver='lbfgs', max_iter=100000)),
               ])
logreg.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 LogisticRegression(C=100000.0, max_iter=100000, n_jobs=1))])

In [38]:
%%time
y_pred = logreg.predict(X_test)

CPU times: user 132 ms, sys: 514 µs, total: 132 ms
Wall time: 137 ms


In [42]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_issue_area))

accuracy 0.7088607594936709
                    precision    recall  f1-score   support

 Economic Activity       0.73      0.77      0.75       108
Criminal Procedure       0.75      0.94      0.84       173
    Judicial Power       0.64      0.69      0.67        98
            Others       0.74      0.79      0.76        71
      Civil Rights       0.56      0.30      0.39        81
   First Amendment       0.69      0.53      0.60       101

          accuracy                           0.71       632
         macro avg       0.69      0.67      0.67       632
      weighted avg       0.70      0.71      0.69       632



In [43]:
y_pred = logreg.predict(df_toPredict["facts"])

In [44]:
y_pred_df = pd.DataFrame({"issue_area": y_pred})

In [45]:
df_toPredict = pd.concat([df_toPredict, y_pred_df], axis = 1)
df_toPredict = df_toPredict[["facts", "issue_area", "first_party_winner"]]

In [47]:
big_df = pd.concat([df_withoutNA, df_toPredict])
big_df = big_df.reset_index(drop=True)

In [49]:
big_df.head(3)

,facts,issue_area,first_party_winner
0,joan stanley three children peter stanley stan...,Civil Rights,1
1,john giglio convict pass forge money order app...,Others,1
2,idaho probate code specify males must prefer f...,Civil Rights,1


# One Hot Encoding


In [50]:
from sklearn.preprocessing import OneHotEncoder

In [51]:
encoder = OneHotEncoder(handle_unknown='ignore')

In [52]:
encoder_df = pd.DataFrame(encoder.fit_transform(big_df[['issue_area']]).toarray())

In [53]:
encoder_df.columns = ['Civil Rights', 'Criminal Procedure', 'Economic Activity',
                      'First Amendment', 'Judicial Power', 'Others']

In [55]:
big_df = big_df.join(encoder_df)
big_df.drop('issue_area', axis=1, inplace=True)

In [56]:
big_df[['Civil Rights', 'Criminal Procedure', 'Economic Activity',
                      'First Amendment', 'Judicial Power', 'Others']] = big_df[['Civil Rights', 'Criminal Procedure', 'Economic Activity',
                      'First Amendment', 'Judicial Power', 'Others']].astype(int)

In [57]:
big_df = big_df[["facts", 'Civil Rights', 'Criminal Procedure', 'Economic Activity',
                      'First Amendment', 'Judicial Power', 'Others', "first_party_winner"]]

In [58]:
big_df

,facts,Civil Rights,Criminal Procedure,Economic Activity,First Amendment,Judicial Power,Others,first_party_winner
0,joan stanley three children peter stanley stan...,1,0,0,0,0,0,1
1,john giglio convict pass forge money order app...,0,0,0,0,0,1,1
2,idaho probate code specify males must prefer f...,1,0,0,0,0,0,1
3,miller conduct mass mail campaign advertise sa...,0,0,0,1,0,0,1
4,ernest mandel belgian professional journalist ...,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...
6283,fair labor standards act violations violations...,0,0,0,0,0,1,1
6284,guidelines regularly test user accessibility b...,1,0,0,0,0,0,0
6285,action bring class action plaintiff bring acti...,0,0,0,0,1,0,1
6286,discovery may reveal transmission additional f...,0,0,0,0,1,0,0


In [62]:
big_df.rename(columns = {'first_party_winner':'target'}, inplace = True)

# Second Prediction with TFIDF

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(big_df, test_size=0.2, random_state=42)

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
vectorizer_tf = TfidfVectorizer(max_df=0.75, max_features=1000, ngram_range=(1,2))

In [69]:
train_vectors = vectorizer_tf.fit_transform(X_train.facts) 
test_vectors = vectorizer_tf.transform(X_test.facts)

In [70]:
train_df=pd.DataFrame(train_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
test_df=pd.DataFrame(test_vectors.toarray(), columns=vectorizer_tf.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [75]:
train_df=pd.concat([train_df,X_train.loc[:, X_train.columns != "facts"].reset_index(drop=True)], axis=1)
test_df=pd.concat([test_df,X_test.loc[:, X_test.columns != "facts"].reset_index(drop=True)], axis=1)

In [89]:
train_df.head(3)

,ability,access,access barriers,access defendant,access facilities,access website,accessibility,accessible,accommodation,accommodations,...,york city,york state,target,Civil Rights,Criminal Procedure,Economic Activity,First Amendment,Judicial Power,Others,target
0,0.038885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,1,1
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,1,1
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,1,0,0,1


In [86]:
X_train = train_df.loc[:, train_df.columns != "target"]
y_train = train_df["target"]

In [94]:
X_test = test_df.loc[:, test_df.columns != "target"]
y_test = test_df["target"]

In [97]:
y_train = y_train.loc[:,~y_train.columns.duplicated()]
y_test = y_test.loc[:,~y_test.columns.duplicated()]

In [106]:
my_target = ["0", "1"]

In [92]:
logreg = Pipeline([
                ('clf', LogisticRegression()),
               ])
logreg.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('clf',
                 LogisticRegression(C=100000.0, max_iter=100000, n_jobs=1))])

In [99]:
%%time
y_pred = logreg.predict(X_test)

CPU times: user 18.4 ms, sys: 9.85 ms, total: 28.2 ms
Wall time: 28.2 ms


In [105]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6033386327503975
              precision    recall  f1-score   support

           0       0.53      0.48      0.51       532
           1       0.65      0.69      0.67       726

    accuracy                           0.60      1258
   macro avg       0.59      0.59      0.59      1258
weighted avg       0.60      0.60      0.60      1258



In [108]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('clf', MultinomialNB())])

In [109]:
%%time
y_pred = logreg.predict(X_test)

CPU times: user 20.9 ms, sys: 386 µs, total: 21.3 ms
Wall time: 24.6 ms


In [110]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6033386327503975
              precision    recall  f1-score   support

           0       0.53      0.48      0.51       532
           1       0.65      0.69      0.67       726

    accuracy                           0.60      1258
   macro avg       0.59      0.59      0.59      1258
weighted avg       0.60      0.60      0.60      1258



In [111]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [112]:
%%time
y_pred = sgd.predict(X_test)

CPU times: user 18.6 ms, sys: 1.91 ms, total: 20.5 ms
Wall time: 21.2 ms


In [113]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6335453100158982
              precision    recall  f1-score   support

           0       0.62      0.35      0.45       532
           1       0.64      0.84      0.73       726

    accuracy                           0.63      1258
   macro avg       0.63      0.60      0.59      1258
weighted avg       0.63      0.63      0.61      1258



In [115]:
from sklearn.ensemble import RandomForestClassifier

randfor = Pipeline([
                ('clf', RandomForestClassifier(n_estimators= 10, max_depth=10, random_state=10)),
               ])
randfor.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('clf',
                 RandomForestClassifier(max_depth=10, n_estimators=10,
                                        random_state=10))])

In [117]:
%%time
y_pred = randfor.predict(X_test)

CPU times: user 30.5 ms, sys: 1.8 ms, total: 32.3 ms
Wall time: 45.5 ms


In [118]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6168521462639109
              precision    recall  f1-score   support

           0       0.59      0.31      0.41       532
           1       0.62      0.84      0.72       726

    accuracy                           0.62      1258
   macro avg       0.61      0.58      0.56      1258
weighted avg       0.61      0.62      0.59      1258



In [119]:
from sklearn.ensemble import GradientBoostingClassifier
graboos = Pipeline([
                ('clf', GradientBoostingClassifier(n_estimators=10, max_depth=10, random_state=10)),
               ])
graboos.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('clf',
                 GradientBoostingClassifier(max_depth=10, n_estimators=10,
                                            random_state=10))])

In [120]:
%%time
y_pred = graboos.predict(X_test)

CPU times: user 25.8 ms, sys: 924 µs, total: 26.7 ms
Wall time: 30.6 ms


In [121]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6057233704292527
              precision    recall  f1-score   support

           0       0.57      0.28      0.38       532
           1       0.62      0.84      0.71       726

    accuracy                           0.61      1258
   macro avg       0.59      0.56      0.54      1258
weighted avg       0.60      0.61      0.57      1258



In [122]:
import xgboost as xgb

xgboo = Pipeline([
                ('clf', xgb.XGBClassifier()),
               ])
xgboo.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('clf', XGBClassifier())])

In [123]:
%%time
y_pred = xgboo.predict(X_test)

CPU times: user 39.1 ms, sys: 8.03 ms, total: 47.2 ms
Wall time: 47.1 ms


In [124]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6216216216216216
              precision    recall  f1-score   support

           0       0.60      0.32      0.42       532
           1       0.63      0.84      0.72       726

    accuracy                           0.62      1258
   macro avg       0.61      0.58      0.57      1258
weighted avg       0.62      0.62      0.59      1258



In [125]:
from sklearn.neighbors import KNeighborsClassifier

kn = Pipeline([
              ('kn',KNeighborsClassifier(n_neighbors=3)),
               ])
kn.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Pipeline(steps=[('kn', KNeighborsClassifier(n_neighbors=3))])

In [126]:
%%time
y_pred = kn.predict(X_test)

CPU times: user 901 ms, sys: 231 ms, total: 1.13 s
Wall time: 920 ms


In [127]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.5930047694753577
              precision    recall  f1-score   support

           0       0.52      0.46      0.49       532
           1       0.64      0.69      0.66       726

    accuracy                           0.59      1258
   macro avg       0.58      0.58      0.58      1258
weighted avg       0.59      0.59      0.59      1258



# Second Prediction with Count

In [128]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(big_df, test_size=0.2, random_state=42)

In [129]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [130]:
train_vectors = vectorizer_tf.fit_transform(X_train.facts) 
test_vectors = vectorizer_tf.transform(X_test.facts)

In [131]:
train_df=pd.DataFrame(train_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
test_df=pd.DataFrame(test_vectors.toarray(), columns=vectorizer_tf.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [132]:
train_df=pd.concat([train_df,X_train.loc[:, X_train.columns != "facts"].reset_index(drop=True)], axis=1)
test_df=pd.concat([test_df,X_test.loc[:, X_test.columns != "facts"].reset_index(drop=True)], axis=1)

In [133]:
X_train = train_df.loc[:, train_df.columns != "target"]
y_train = train_df["target"]
X_test = test_df.loc[:, test_df.columns != "target"]
y_test = test_df["target"]

In [142]:
logreg = Pipeline([
                ('clf', LogisticRegression()),
               ])
logreg.fit(X_train, y_train)

Pipeline(steps=[('clf', LogisticRegression())])

In [143]:
%%time
y_pred = logreg.predict(X_test)

CPU times: user 20.9 ms, sys: 0 ns, total: 20.9 ms
Wall time: 24.9 ms


In [144]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6343402225755167
              precision    recall  f1-score   support

           0       0.60      0.41      0.49       532
           1       0.65      0.80      0.72       726

    accuracy                           0.63      1258
   macro avg       0.62      0.60      0.60      1258
weighted avg       0.63      0.63      0.62      1258



In [141]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

Pipeline(steps=[('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [145]:
%%time
y_pred = sgd.predict(X_test)

CPU times: user 23.7 ms, sys: 1.93 ms, total: 25.7 ms
Wall time: 33.3 ms


In [146]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6335453100158982
              precision    recall  f1-score   support

           0       0.62      0.35      0.45       532
           1       0.64      0.84      0.73       726

    accuracy                           0.63      1258
   macro avg       0.63      0.60      0.59      1258
weighted avg       0.63      0.63      0.61      1258



In [147]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('clf', MultinomialNB())])

In [149]:
%%time
y_pred = nb.predict(X_test)

CPU times: user 32 ms, sys: 772 µs, total: 32.8 ms
Wall time: 45.4 ms


In [150]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6248012718600954
              precision    recall  f1-score   support

           0       0.58      0.40      0.47       532
           1       0.64      0.79      0.71       726

    accuracy                           0.62      1258
   macro avg       0.61      0.59      0.59      1258
weighted avg       0.62      0.62      0.61      1258



In [151]:
from sklearn.ensemble import RandomForestClassifier

randfor = Pipeline([
                ('clf', RandomForestClassifier(n_estimators= 10, max_depth=10, random_state=10)),
               ])
randfor.fit(X_train, y_train)

Pipeline(steps=[('clf',
                 RandomForestClassifier(max_depth=10, n_estimators=10,
                                        random_state=10))])

In [152]:
%%time
y_pred = randfor.predict(X_test)

CPU times: user 31.6 ms, sys: 0 ns, total: 31.6 ms
Wall time: 79.8 ms


In [153]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6168521462639109
              precision    recall  f1-score   support

           0       0.59      0.31      0.41       532
           1       0.62      0.84      0.72       726

    accuracy                           0.62      1258
   macro avg       0.61      0.58      0.56      1258
weighted avg       0.61      0.62      0.59      1258



In [154]:
from sklearn.ensemble import GradientBoostingClassifier
graboos = Pipeline([
                ('clf', GradientBoostingClassifier(n_estimators=10, max_depth=10, random_state=10)),
               ])
graboos.fit(X_train, y_train)

Pipeline(steps=[('clf',
                 GradientBoostingClassifier(max_depth=10, n_estimators=10,
                                            random_state=10))])

In [155]:
%%time
y_pred = graboos.predict(X_test)

CPU times: user 33 ms, sys: 488 µs, total: 33.4 ms
Wall time: 38.2 ms


In [156]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6057233704292527
              precision    recall  f1-score   support

           0       0.57      0.28      0.38       532
           1       0.62      0.84      0.71       726

    accuracy                           0.61      1258
   macro avg       0.59      0.56      0.54      1258
weighted avg       0.60      0.61      0.57      1258



In [157]:
import xgboost as xgb

xgboo = Pipeline([
                ('clf', xgb.XGBClassifier()),
               ])
xgboo.fit(X_train, y_train)

Pipeline(steps=[('clf', XGBClassifier())])

In [158]:
%%time
y_pred = xgboo.predict(X_test)

CPU times: user 49.7 ms, sys: 6.91 ms, total: 56.6 ms
Wall time: 64.9 ms


In [159]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.6216216216216216
              precision    recall  f1-score   support

           0       0.60      0.32      0.42       532
           1       0.63      0.84      0.72       726

    accuracy                           0.62      1258
   macro avg       0.61      0.58      0.57      1258
weighted avg       0.62      0.62      0.59      1258



In [169]:
from sklearn.neighbors import KNeighborsClassifier

kn = Pipeline([
              ('kn',KNeighborsClassifier(n_neighbors=15)),
               ])
kn.fit(X_train, y_train)

Pipeline(steps=[('kn', KNeighborsClassifier(n_neighbors=15))])

In [170]:
%%time
y_pred = kn.predict(X_test)

CPU times: user 957 ms, sys: 145 ms, total: 1.1 s
Wall time: 633 ms


In [171]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_target))

accuracy 0.609697933227345
              precision    recall  f1-score   support

           0       0.56      0.38      0.45       532
           1       0.63      0.78      0.70       726

    accuracy                           0.61      1258
   macro avg       0.59      0.58      0.58      1258
weighted avg       0.60      0.61      0.59      1258

